<a href="https://colab.research.google.com/github/bharathssa/Machine-Learning/blob/main/Cardekho%20selling%20price%20prediction_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [226]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [227]:
df = pd.read_csv('/kaggle/input/cardekho/car data.csv')

In [228]:
df.info()

In [229]:
### There is no null value
df.head()

In [230]:
df.describe()

In [231]:
df.head()

In [232]:
from datetime import date
today = date.today()
current_year = today.year
print(current_year)

In [233]:
df['current_year'] = current_year
df.head(5)

In [234]:
df['No. of Years'] = df['current_year'] - df['Year']
df.head(3)

In [235]:
# Year and Current year is not required at this stage since we have calculate the no.of year of the car

df.drop(columns = ['Year','current_year'], inplace= True, axis =1 )

df.head(3)

Let's Do some EDA

In [236]:
numerical_features = [i for i in df.columns if df.dtypes[i]!='O']
print(numerical_features)

In [237]:
categorical_features = [i for i in df.columns if df.dtypes[i]=='O']
print (categorical_features)

In [238]:
### Extracting Unique features from categorical_features
for i in categorical_features:
  print ('{}:'.format(i) , df[i].unique())


In [239]:
### Let's see count of Fuel_varient used accross various car model
df['Car_Name'].groupby(df['Fuel_Type']).count().sort_values(ascending=False)

In [240]:
### Let's see count of Transmission used accross various car model

df['Car_Name'].groupby(df['Transmission']).count().sort_values(ascending=False)

In [241]:
### Let's see count of Seller_type used accross various car model

df['Car_Name'].groupby(df['Seller_Type']).count().sort_values(ascending=False)

In [242]:
sns.pairplot(df)

In [243]:
### Top 5 Model using Fuel_Type

plt.figure (figsize=(10,7))
sns.countplot(y='Car_Name', hue='Fuel_Type', data=df, order=df['Car_Name'].value_counts().iloc[:5].index)

In [244]:
### Top 5 Model using Transmission

plt.figure (figsize=(10,7))
sns.countplot(y='Car_Name', hue='Transmission', data=df, order=df['Car_Name'].value_counts().iloc[:5].index)

In [245]:
###LEt's see Top 10 car model's selling price using Fuel_type
fig, ax = plt.subplots(figsize=(15,6))
ax = sns.barplot(y='Selling_Price', x='Car_Name', data=df, hue='Fuel_Type', order=df['Car_Name'].value_counts().iloc[:10].index)

In [246]:
### Let's see whether Kms_Driven affects selling Price
fig, ax = plt.subplots(figsize=(15,6))
ax = sns.barplot(y='Selling_Price', x='Kms_Driven', data=df, order=df['Kms_Driven'].sort_values(ascending=True).value_counts().iloc[:10].index )

In [247]:
###Box plot to find outliers

fig, ax = plt.subplots(figsize=(20,3))
ax = sns.boxplot(x="Kms_Driven", y="Selling_Price", data=df, order=df['Kms_Driven'].sort_values(ascending=True).value_counts().iloc[:10].index)

In [248]:
print(df.corr())

In [249]:
dataplot = sns.heatmap( df.corr(), cmap="YlGnBu", annot= True )
plt.show()

### Feature Engineering Part

In [250]:
df.plot(kind='box')

In [251]:
sns.boxplot(df['Kms_Driven'])

In [252]:
print(np.where(df['Kms_Driven']>400000))

In [253]:
from scipy import stats
z = np.abs(stats.zscore(df['Kms_Driven']))
print(z)

In [254]:
Threshold = 3
z1=np.where(z>3)
z2=np.array(z1)
print(z2)
z3 = z2.flatten()
print(z3)

In [255]:
print('Old Shape:', df.shape)

In [256]:
df.drop(z3, inplace=True, axis=0)
df

In [257]:
print('New Shape:',df.shape)

### Feature Selection

In [258]:
df.drop(['Car_Name'], inplace=True, axis=1)


In [259]:
df.head(10)

In [260]:
df = pd.get_dummies(df, drop_first=True)
df

In [261]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [262]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5)

In [263]:
print(X_train.shape)
print(X_test.shape)

In [264]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators=125)

In [265]:
reg.fit(X_train, y_train)

In [266]:
y_pred = reg.predict(X_test)
y_pred

In [267]:
from sklearn.metrics import mean_squared_error

In [268]:
print('Mean Squared Error:',mean_squared_error(y_test, y_pred))


In [269]:
###Hyperparameter tuning (Random Search)
import xgboost as xgb
# from itertools import product
# import random

# #params
# booster = 'gbtree'
# silent = 0

# eta = [0.01,0.1,0.2,0.3]  #learning rate in gbm
# min_child_weight = [1,2,0.5]  #minimum sum of weights of all observations required in a child
# max_depth = [3,6,8]  #maximum depth of the tree
# gamma = [0,1]  #a split will occur only when reduction in loss function > gamma 
# subsample = [0.5,0.7,1] #fraction of rows to be sampled to make a decision tree
# colsample_bytree = [0.5,0.7,1] #fraction of columns to be sampled to make a decision tree
# # lambda_ = [0.4,0.8,1] #L2 regularization weights
# alpha = [0,1]  #L1 reg weights



# params =  [eta,min_child_weight,max_depth,gamma,subsample,colsample_bytree,alpha]
# parameters = list(product(*params))
# parameters = random.sample(parameters,100)
# len(parameters)


# min_rmse = float('inf')
# best_params = parameters[0]
# for p in parameters:
#     eta = p[0]
#     min_child_weight = p[1]
#     max_depth = p[2]
#     gamma = p[3]
#     subsample = p[4]
#     colsample_bytree = p[5]
#     alpha = p[6]

#     print('Random Search On Hyperparamters Xgboost')
#     print(f"Fitting params -> eta: {eta}, min_child_weight: {min_child_weight} ,\
#     max_depth:{max_depth}, gamma: {gamma}, subsample : {subsample},\
#     col_sample_bytree: {colsample_bytree:},alpha: {alpha}")
    
#     model = xgb.XGBRegressor(random_state=42,booster = 'gbtree',verbosity=0,eta=eta,
#                              min_child_weight=min_child_weight,max_depth=max_depth,gamma=gamma,
#                              subsample=subsample,colsample_bytree=colsample_bytree,alpha=alpha)
    
#     model.fit(X_train,y_train)
#     y_pred = reg.predict(X_test)
    
#     y_pred_rmse = mean_squared_error(y_test,y_pred,squared=False)
    
    
#     print(f'Train rmse: {y_pred_rmse}')
    
    
#     if y_pred_rmse < min_rmse:
#         min_rmse = y_pred_rmse
#         best_params = p
        
#     print('--'*50)
    

    
# print(f'Found following best parameters: eta: {best_params[0]},\
# min_child_weight: {best_params[1]} , max_depth:{best_params[2]}, gamma: {best_params[3]}, subsample : {best_params[4]}, \
# col_sample_bytree: {best_params[5]},alpha: {best_params[6]} with validation loss {min_rmse}')

In [270]:
import xgboost as xgb
eta = 0.03
min_child_weight = 0.5
max_depth = 6
gamma = 1
subsample  = 0.5
colsample_bytree = 0.7
alpha = 0

model = xgb.XGBRegressor(random_state=5,booster = 'gbtree',verbosity=0,eta=eta,min_child_weight=min_child_weight,
                         max_depth=max_depth,gamma=gamma,subsample=subsample,colsample_bytree=colsample_bytree,alpha=alpha)
model.fit(X_train, y_train)
y_pred = reg.predict(X_test)

y_pred_rmse = mean_squared_error(y_test,y_pred,squared=False)


print('Mean Squared Error:',y_pred_rmse)

